In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
import logging
import pickle

In [ ]:
log_fmt = "[%(asctime)s] %(levelname)s in %(module)s: %(message)s"
logging.basicConfig(format=log_fmt, level=logging.INFO)

In [ ]:
base_path = './data'
feature_path = './feature'

In [ ]:
train = pd.read_csv(f'{base_path}/invite_info_0926.txt', sep='\t', header=None)
train.columns = ['qid', 'uid', 'dt', 'label']

del train['dt']
logging.info("train %s", train.shape)

test = pd.read_csv(f'{base_path}/invite_info_evaluate_0926.txt', sep='\t', header=None)
test.columns = ['qid', 'uid', 'dt']
sub = test.copy()
sub_size = len(sub)

del test['dt']
logging.info("test %s", test.shape)

In [ ]:
# 加载 ans kfold feature
# cols = ['day', 'hour', 'q_inv_kfold_mean', 'q_inv_kfold_sum', 'q_inv_kfold_std', 'q_inv_kfold_count', 
#          'u_inv_kfold_mean', 'u_inv_kfold_sum', 'u_inv_kfold_std', 'u_inv_kfold_count', 
#          'q_ans_kfold_count', 'u_ans_kfold_count', 'q_diff_qa_days_sum', 'q_diff_qa_days_max', 
#          'q_diff_qa_days_mean', 'u_diff_qa_days_sum', 'u_diff_qa_days_max', 'u_diff_qa_days_mean']

# t1 = pd.read_csv(f'{feature_path}/train_kfold_feature.txt', sep='\t', usecols=cols)
t1 = pd.read_csv(f'{feature_path}/train_kfold_feature.txt', sep='\t')
train = pd.concat([train, t1], axis=1)
logging.info("train %s", train.shape)

# t1 = pd.read_csv(f'{feature_path}/test_kfold_feature.txt', sep='\t', usecols=cols)
t1 = pd.read_csv(f'{feature_path}/test_kfold_feature.txt', sep='\t')
test = pd.concat([test, t1], axis=1)
logging.info("test %s", test.shape)

In [ ]:
train['week'] = train['day']%7
test['week'] = test['day']%7

In [ ]:
# 加载 invete feature 1
t1 = pd.read_csv(f'{feature_path}/train_invite_feature.txt', sep='\t')
train = pd.concat([train, t1], axis=1)
logging.info("train %s", train.shape)

t1 = pd.read_csv(f'{feature_path}/test_invite_feature.txt', sep='\t')
test = pd.concat([test, t1], axis=1)
logging.info("test %s", test.shape)

In [ ]:
# 加载 invete feature 2
t1 = pd.read_csv(f'{feature_path}/train_invite_feature_2.txt', sep='\t')
train = pd.concat([train, t1], axis=1)
logging.info("train %s", train.shape)

t1 = pd.read_csv(f'{feature_path}/test_invite_feature_2.txt', sep='\t')
test = pd.concat([test, t1], axis=1)
logging.info("test %s", test.shape)

In [ ]:
# 加载 kfold topic feature, QU
t1 = pd.read_csv(f'{feature_path}/train_kfold_topic_feature.txt', sep='\t')
train = pd.concat([train, t1], axis=1)
logging.info("train %s", train.shape)

t1 = pd.read_csv(f'{feature_path}/test_kfold_topic_feature.txt', sep='\t')
test = pd.concat([test, t1], axis=1)
logging.info("test %s", test.shape)

In [ ]:
# 加载 user kfold topic feature，UU
t1 = pd.read_csv(f'{feature_path}/train_kfold_ut_feature.txt', sep='\t')
train = pd.concat([train, t1], axis=1)
logging.info("train %s", train.shape)

t1 = pd.read_csv(f'{feature_path}/test_kfold_ut_feature.txt', sep='\t')
test = pd.concat([test, t1], axis=1)
logging.info("test %s", test.shape)

In [ ]:
# 加载 kfold label 特征
t1 = pickle.load(open(f'{feature_path}/train_kfold_label_feature.pkl', 'rb'))
train = pd.concat([train, t1], axis=1)
logging.info("train %s", train.shape)

t1 = pickle.load(open(f'{feature_path}/test_kfold_label_feature.pkl', 'rb'))
test = pd.concat([test, t1], axis=1)
logging.info("test %s", test.shape)
# single_targets = ['uid', 'qid', 'freq', 'score', 
#                   'uf_b1', 'uf_b2','uf_b3', 'uf_b4', 'uf_b5', 
#                   'uf_c1', 'uf_c2', 'uf_c3', 'uf_c4', 'uf_c5',
#                   'diff_iq_day', 'diff_iq_hour', 
#                   'intersection_ft_count', 'intersection_it_count']
# for f in single_targets:
#     logging.info('adding kfold label feature, at: %s', f)
    
#     t1 = pickle.load(open(f'{feature_path}/single_kfold_feat/train_{f}.pkl', 'rb'))
#     train = pd.concat([train, t1], axis=1)
    
#     t1 = pickle.load(open(f'{feature_path}/single_kfold_feat/test_{f}_merged.pkl', 'rb'))
#     test = pd.concat([test, t1], axis=1)

In [ ]:
train

In [ ]:
test[1141680:1141688]

In [ ]:
test

In [ ]:
# 加载用户
user = pd.read_csv(f'{base_path}/member_info_0926.txt', header=None, sep='\t')
user.columns = ['uid', 'gender', 'freq', 'uf_b1', 'uf_b2','uf_b3', 'uf_b4', 'uf_b5', 
                'uf_c1', 'uf_c2', 'uf_c3', 'uf_c4', 'uf_c5',  'score', 'follow_topic', 'inter_topic']

del user['follow_topic'], user['inter_topic']
logging.info("user %s", user.shape)

unq = user.nunique()
logging.info("user unq %s", unq)

for x in unq[unq == 1].index:
    del user[x]
    logging.info('del unq==1 %s', x)

t = user.dtypes
cats = [x for x in t[t == 'object'].index if x not in ['follow_topic', 'inter_topic', 'uid']]
logging.info("user cat %s", cats)

for d in cats:
    lb = LabelEncoder()
    user[d] = lb.fit_transform(user[d])
    logging.info('encode %s', d)
    
logging.info('encoding qid...')    
q_lb = LabelEncoder()
q_lb.fit(list(train['qid'].astype(str).values) + list(test['qid'].astype(str).values))
train['qid_enc'] = q_lb.transform(train['qid'])
test['qid_enc'] = q_lb.transform(test['qid'])
logging.info('add qid_enc')

logging.info('encoding uid...')
u_lb = LabelEncoder()
u_lb.fit(user['uid'])
train['uid_enc'] = u_lb.transform(train['uid'])
test['uid_enc'] = u_lb.transform(test['uid'])
logging.info('add uid_enc')

# merge user
train = pd.merge(train, user, on='uid', how='left')
test = pd.merge(test, user, on='uid', how='left')
logging.info("train shape %s, test shape %s", train.shape, test.shape)

In [ ]:
data = pd.concat((train, test), axis=0, sort=True)
len_train = len(train)
del train

In [ ]:
# count 特征
count_fea = ['uid_enc', 'qid_enc', 'gender', 'freq', 'uf_c1', 'uf_c2', 'uf_c3', 'uf_c4', 'uf_c5']
for feat in count_fea:
    logging.info('counting %s', fea)
    col_name = '{}_count'.format(feat)
    data[col_name] = data[feat].map(data[feat].value_counts().astype(int))
    data.loc[data[col_name] < 2, feat] = -1
    data[feat] += 1
    data[col_name] = data[feat].map(data[feat].value_counts().astype(int))
    data[col_name] = (data[col_name] - data[col_name].min()) / (data[col_name].max() - data[col_name].min())

In [ ]:
drop_feat = ['label', 'uid', 'qid', 'dt', 'day']
drop_feat += ['u_is_rec_mean', 'u_reci_uncheer_mean', 'q_is_dest_sum', 'u_reci_uncheer_sum', 'u_is_rec_max', 
             'u_is_dest_mean','q_reci_uncheer_mean', 'q_reci_uncheer_sum', 'u_is_dest_sum', 'q_is_dest_max',
             'q_reci_uncheer_max', 'u_reci_tks_max', 'q_reci_mark_max','u_reci_dis_max', 'q_has_video_mean',
             'q_reci_no_help_mean', 'count_u_topic', 'u_has_video_mean', 'q_reci_dis_sum', 'q_reci_mark_sum',
             'q_reci_tks_sum','q_reci_tks_max','q_reci_dis_max','u_reci_mark_max','q_is_good_mean',
             'q_reci_no_help_sum', 'q_reci_xxx_max', 'u_reci_xxx_max','u_reci_no_help_sum','u_reci_xxx_sum',
              'u_is_good_mean','q_reci_no_help_max','u_has_img_max','u_is_good_sum','u_reci_no_help_max',
              'u_has_video_sum','uf_b5','q_reci_xxx_sum','q_is_good_sum','q_has_img_max','q_has_video_sum',
              'q_has_video_max','u_has_video_max','q_is_good_max','q_is_rec_max','u_is_good_max',
              'q_is_dest_mean','u_reci_uncheer_max','uf_c5_count','u_is_dest_max','q_is_rec_mean',
              'q_is_rec_sum','u_is_rec_sum', 'q_reci_xxx_mean','u_reci_xxx_mean','u_reci_comment_max',
              'q_reci_comment_sum','u_reci_cheer_max','u_reci_dis_sum','u_reci_tks_sum','q_has_img_sum',
              'q_reci_comment_max','q_reci_cheer_max','u_reci_no_help_mean','u_has_img_sum','u_reci_mark_sum']
# drop_feat += ['q_is_good_sum', 'q_is_good_max', 'q_is_good_mean', 'u_is_good_sum', 'u_is_good_max', 
#               'u_is_good_mean', 'q_is_rec_sum', 'q_is_rec_max', 'q_is_rec_mean', 'u_is_rec_sum', 
#               'u_is_rec_max', 'u_is_rec_mean', 'q_is_dest_sum', 'q_is_dest_max', 'q_is_dest_mean', 
#               'u_is_dest_sum', 'u_is_dest_max', 'u_is_dest_mean', 'q_has_img_sum', 'q_has_img_max', 
#               'q_has_img_mean', 'u_has_img_sum', 'u_has_img_max', 'u_has_img_mean', 'q_has_video_sum', 
#               'q_has_video_max', 'q_has_video_mean', 'u_has_video_sum', 'u_has_video_max', 'u_has_video_mean', 
#               'q_word_count_sum', 'q_word_count_max', 'q_word_count_mean', 'u_word_count_sum', 
#               'u_word_count_max', 'u_word_count_mean', 'q_reci_cheer_sum', 'q_reci_cheer_max', 
#               'q_reci_cheer_mean', 'u_reci_cheer_sum', 'u_reci_cheer_max', 'u_reci_cheer_mean', 
#               'q_reci_uncheer_sum', 'q_reci_uncheer_max', 'q_reci_uncheer_mean', 'u_reci_uncheer_sum',
#               'u_reci_uncheer_max', 'u_reci_uncheer_mean', 'q_reci_comment_sum', 'q_reci_comment_max', 
#               'q_reci_comment_mean', 'u_reci_comment_sum', 'u_reci_comment_max', 'u_reci_comment_mean', 
#               'q_reci_mark_sum', 'q_reci_mark_max', 'q_reci_mark_mean', 'u_reci_mark_sum', 'u_reci_mark_max', 
#               'u_reci_mark_mean', 'q_reci_tks_sum', 'q_reci_tks_max', 'q_reci_tks_mean', 'u_reci_tks_sum', 
#               'u_reci_tks_max', 'u_reci_tks_mean', 'q_reci_xxx_sum', 'q_reci_xxx_max', 'q_reci_xxx_mean', 
#               'u_reci_xxx_sum', 'u_reci_xxx_max', 'u_reci_xxx_mean', 'q_reci_no_help_sum', 'q_reci_no_help_max',
#               'q_reci_no_help_mean', 'u_reci_no_help_sum', 'u_reci_no_help_max', 'u_reci_no_help_mean', 
#               'q_reci_dis_sum', 'q_reci_dis_max', 'q_reci_dis_mean', 'u_reci_dis_sum', 'u_reci_dis_max', 
#               'u_reci_dis_mean']

feature_cols = [x for x in data.columns if x not in drop_feat]
# feature_cols

In [ ]:
feature_cols

In [ ]:
logging.info("feature size %s", len(feature_cols))

X_train_all = data.iloc[:len_train][feature_cols]
y_train_all = data.iloc[:len_train]['label']
X_test = data.iloc[len_train:]
assert len(X_test) == sub_size

fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for index, (train_idx, val_idx) in enumerate(fold.split(X=X_train_all, y=y_train_all)):
    break

X_train, X_val, y_train, y_val = X_train_all.iloc[train_idx][feature_cols], X_train_all.iloc[val_idx][feature_cols], \
                                 y_train_all.iloc[train_idx], \
                                 y_train_all.iloc[val_idx]
del X_train_all

logging.info("train shape %s, val shape %s, test shape %s", X_train.shape, X_val.shape, X_test.shape)

model_lgb = LGBMClassifier(n_estimators=2000, n_jobs=-1, objective='binary', seed=1000, silent=True)
model_lgb.fit(X_train, y_train,
              eval_metric=['logloss', 'auc'],
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=50)



In [ ]:
sub['label'] = model_lgb.predict_proba(X_test[feature_cols])[:, 1]
sub.to_csv('./result/2000_0.877402.txt', index=None, header=None, sep='\t')

In [ ]:
fi = pd.DataFrame({'feature': feature_cols, 'imp': model_lgb.feature_importances_})
fi['rate'] = fi['imp'] / fi['imp'].sum()
fi_sorted = fi.sort_values(by='rate', ascending=False)

In [ ]:
# import pickle
# pickle.dump(fi_sorted, open('./feature_importance.pkl', 'wb'))

In [ ]:
fi_sorted[-60：]

In [ ]:
temp = []
for i in list(fi_sorted[-80:-60].feature):
    temp.append(i)
temp